# Is this a picture of a puppy or a bagel? Let a convolutional neural network (CNN) tell us

CNNs are computer vision models that look at many pictures (turned into numeric values) and classify what the pictures represent based on those values. CNNs iterate through many pictures and learn the proper classification using some kind of performance measure (i.e. training a model). 

### step 1: import our libraries
* numpy and pandas for matrices and text data
* tensorflow for our neural net

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow
from tensorflow import keras
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Rescaling, Conv2D, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.applications import resnet50, mobilenet
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import image_dataset_from_directory, to_categorical
from tensorflow.keras.applications.xception import preprocess_input
import cv2
import os

### step 2: define the parameters of our learning algorithm
* batch sizes represent the number of samples the model will work through before updating its learning 
* epochs represent the number of whole passes through the training data
* CNNs use the training data to learn (also split into validation data to grade its performance)
* Test data is a batch of unseen pictures we run through a trained model
* the learning rate is a way to optimize how well the model learns during each epoch

In [ ]:
EPOCHS = 20
IMGSIZE = 300
MODEL_NAME = 'cnn_imgsize300'
OPTIMIZER = 'adam'
TRAINING_DIR = '../input/bagel-or-puppy/bagel_puppy/training_set/'
TEST_DIR = '../input/bagel-or-puppy/bagel_puppy/test_set/'

In [ ]:
early_stopping = tensorflow.keras.callbacks.EarlyStopping(
    patience = 18,
    min_delta = 0.0005,
    restore_best_weights = True,
    )

filepath = MODEL_NAME+"_bestweights.hdf5"
checkpoint_callback = ModelCheckpoint(filepath, monitor = 'val_acc', save_best_only = True, mode = 'max')

# learning rate schedule for fine tuning 
def exponential_lr(epoch, 
                   epochs = EPOCHS,
                   start_lr = 0.0001, min_lr = 0.0001, max_lr = 0.001,
                   rampup_epochs = 3, sustain_epochs = 8,
                   exp_decay = 0.9):

    def lr(epoch, epochs, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # cos decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
            #lr = 0.5 * max_lr * (
            #1 + np.cos(np.pi * (epoch - rampup_epochs - sustain_epochs) /
            #    float(epochs - rampup_epochs - sustain_epochs))
            #)
        return lr
    return lr(epoch,
              epochs,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs)

lr_callback = tensorflow.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

### step 3: create our training, validation, and test datasets
* we've loaded a bunch of images of cats and dogs into respective folders
* the keras package in Python handles the preprocessing for us: sizes all images to the same dimensions, assigns the proper classes

In [ ]:
training_set = image_dataset_from_directory(TRAINING_DIR, 
                                                 image_size = (IMGSIZE, IMGSIZE),                      
                                                 batch_size = 9,
                                                 label_mode = 'categorical',
                                                 subset = 'training',
                                                 validation_split = 0.2,
                                                 seed =  888
                                                )

validation_set = image_dataset_from_directory(TRAINING_DIR, 
                                                 image_size = (IMGSIZE, IMGSIZE), 
                                                 batch_size = 9,
                                                 label_mode = 'categorical',
                                                 subset = 'validation',
                                                 validation_split = 0.2,
                                                 seed =  888
                                                  ) # set as validation data

test_set = image_dataset_from_directory(TEST_DIR, 
                                        image_size = (IMGSIZE, IMGSIZE), 
                                        batch_size = 3
                                       )

classes = training_set.class_names

### what our training data looks like

In [ ]:
plt.figure(figsize = (10, 10))
for images, labels in training_set.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(classes[np.argmax(labels[i])])
    plt.axis('off')


### step 4: building a model using transfer learning
transfer learning involves using a base model (in this case the xception model) that was developed for another task as our starting point for this task. Xception is a model that was trained to recognize about 1000 different objects, not just bagels and dogs. 

In [ ]:
# the RGB channel values are in the [0, 255] range-not ideal for a neural network
# we want to make our input values small
# standardize values to be in the [0, 1] range by using tf.keras.layers.Rescaling:
# two possible ways: mapping
# training_set = training_set.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(training_set))
# first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

# or include the layer inside the model definition to simplify deployment

In [ ]:
# performance enhancers:
# keep the images in memory after they're loaded off disk during the first epoch with cache()
# overlap data preprocessing and model execution while training
training_set = training_set.cache().prefetch(buffer_size = AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
# pre-trained base (this is transfer learning)
xnet = keras.applications.xception.Xception(
    weights = 'imagenet',
    include_top = False ,
    input_shape = (IMGSIZE, IMGSIZE, 3)
)
xnet.trainable = False

# base & head 
model = keras.Sequential([
    xnet,
    layers.Rescaling(1./255),
    layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', input_shape = (IMGSIZE, IMGSIZE, 3)),  
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(256, activation = 'relu'),
    layers.Dense(2, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

### step 5: train and evaluate the model
we will evaulate performance by looking at the loss function (how well the model's learning went) and the classification accuracy on the validation set

we will also explore a popular method for increasing the performance of CNNs: data augmentation

In [ ]:
history = model.fit(training_set, 
                    epochs = EPOCHS, 
                    validation_data = validation_set, 
                    callbacks = [early_stopping, checkpoint_callback, lr_callback]
                   )

In [ ]:
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize = (10,10), facecolor = '#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)

display_training_curves(
    history.history['accuracy'],
    history.history['val_accuracy'],
    'accuracy',
    212,
)

### see that big gap between our training set accuracy and validation set accuracy? our model is overfitting

when there are a small number of training examples, the model sometimes learns from unwanted details (noise) in those training examples, to the extent it negatively affects the performance of the model on the test data.

let's see if augmentation, i.e. adding additional training data by randomly transforming the images we already have in our training set, helps improve performance

In [ ]:
# define transformations
data_augmentation = keras.Sequential(
  [layers.RandomZoom(0.2),
   layers.RandomFlip(input_shape = (IMGSIZE, IMGSIZE, 3)),
   layers.RandomRotation(0.5, fill_mode = 'constant', fill_value = 1),
  ]
)

training_set_aug = training_set.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
# visualize transformations
plt.figure(figsize = (10, 10))
for images, _ in training_set_aug.take(7):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype('uint8'))
        plt.axis('off')


In [ ]:
history = model.fit(training_set_aug, 
                    epochs = EPOCHS, 
                    validation_data = validation_set, 
                    callbacks = [early_stopping, checkpoint_callback, lr_callback]
                   )

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)

display_training_curves(
    history.history['accuracy'],
    history.history['val_accuracy'],
    'accuracy',
    212,
)

### step 6: running test data through our trained model to see how well it can classify new images of cats and dogs


In [ ]:
# save the model using the weights that yielded the highest accuracy
model.save(filepath)
model_best = load_model(filepath, compile = True)

In [ ]:
predictions = model_best.predict(test_set, 
                                 verbose = 0,
                                 callbacks = None)

In [ ]:
predicted_class_indices = np.argmax(predictions, axis = -1)
predicted_class_confidence = 100*np.max(predictions, axis = -1)

In [ ]:
labels = [0, 1]
names = (test_set.class_names)
labels = dict(zip(labels, names))
predicted_names = [names[k] for k in predicted_class_indices]

In [ ]:
filenames = [os.path.join(root, name)
             for root, dirs, files in os.walk(TEST_DIR)
                 for name in files
                 if name.endswith('.jpg')]
filenames.sort()

true_labels = [os.path.split(os.path.dirname(file))[-1] for file in filenames]

test_df = pd.DataFrame({
    'filename': filenames,
    'true_name': true_labels,
    'predicted_name': predicted_names,
    '% confidence ': predicted_class_confidence
})
pd.set_option('display.max_colwidth', None)
test_df.head(20)

In [ ]:
sample_correct = test_df.query('true_name == predicted_name').sample(n = 6, replace = False)
plt.subplots(2, 3, figsize = (18, 8))

for i in range(len(sample_correct)):
    plt.subplot(2, 3, i + 1)
    plt.axis('Off')
    image = img.imread(sample_correct.iloc[i][0])
    plt.imshow(image)
    plt.title(f'label: {sample_correct.iloc[i][2]}\n confidence: {sample_correct.iloc[i][3]:.3f}%', fontsize = 12);

In [ ]:
sample_incorrect = test_df.query('true_name != predicted_name').sample(n = 6, replace = False)
plt.subplots(2, 3, figsize = (12, 12))

for i in range(len(sample_incorrect)):
    plt.subplot(2, 3, i + 1)
    plt.axis('Off')
    image = img.imread(sample_incorrect.iloc[i][0])
    plt.imshow(image)
    plt.title(f'label: {sample_incorrect.iloc[i][2]}\n confidence: {sample_incorrect.iloc[i][3]:.3f}%', fontsize = 16);

In [ ]:
'''
filenames = [os.path.join(root, name)
             for root, dirs, files in os.walk(TEST_DIR)
                 for name in files
                 if name.endswith('.jpg')]

import PIL.Image
PIL.Image.open(str(filenames[0]))
'''
